# Python

## O duplo significado de uma linguagem interpretada

Que Python é uma linguagem de programação, já sabemos. Enquanto linguagem, Python possui um conjunto bem-definido de regras: léxicas, sintáticas, semânticas e de lógica abstrata.

Porém, algo que em cursos introdutórios por vezes é dito, mas nem sempre explorado, é que Python é uma linguagem interpretada. Isso significa que Python é mais que apenas uma linguagem: ele é também um programa. Inclusive, usaremos aqui a palavra Python com esse duplo significado. Quando clicamos no executável chamado ```python3``` no computador, ou o executamos do terminal, ou ainda deixamos uma IDE ou interface executá-lo por nós, estamos rodando tal programa. E o nome desse programa é **interpretador**. Aqui, estamos por enquanto usando a palavra interpretador em sentido amplo, como todo o ciclo deste robusto programa que permite executar código na linguagem Python.

Há dois jeitos primordiais de iniciar o interpretador: por meio do modo interativo, que nos dá um prompt onde podemos digitar comandos em Python, apertar ```Enter```, e vê-los sendo executados (ou receber um erro); e por meio de scripts, onde especificamos um arquivo ```.py```, usando o comando ```python3 [arquivo].py```, e o interpretador executa a totalidade daquele arquivo de uma vez. Há diversas opções que podem ser configuradas ao invocar o interpretador pelo terminal para customizar seu comportamento, tal como consultar a versão instalada, obter ajuda, passar argumentos ao código a ser executado, entre outros. Não os abordaremos aqui, mas eles podem ser consultados [na documentação oficial](https://docs.python.org/3/using/cmdline.html#using-on-general).

Ocorre que não existe apenas um único programa que é capaz de executar Python. Existem vários. A cada um desses damos o nome de **implementação**. A primeira e mais importante implementação de Python, usada como principal referência e cujo funcionamento é, por vezes, até mesmo mencionado na documentação oficial da própria linguagem, é escrita em C - ou seja, seu interpretador é um programa compilado a partir de instruções escritas em C. É chamada de **CPython**. Existem outras implementações em outras linguagens, como Jython em Java e PyPy em próprio Python(!), ou apenas para permitir a execução da linguagem em outros ambientes ou com outras arquiteturas. Ao percorrer os deste material, você perceberá que algumas vezes o comportamento da linguagem depende da implementação. Vamos nos basear principalmente em CPython, embora eu tente deixar explícito sempre que algum assunto dependa desta implementação em particular.

No restante do presente tópico, vamos pincelar um pouco do funcionamento interno do interpretador, tentando manter a discussão abstrata o suficiente para se aplicar a qualquer implementação.

## O ciclo REPL

O que chamamos acima de interpretador executa uma série de tarefas necessárias para a linguagem chegar do nível de texto compreendido por seres humanos até o resultado ou efeito de um cálculo computacional no nível da máquina. Ao alimentarmos um trecho de código ao interpretador no modo interativo, ele dispara o próximo passo de um ciclo abstrato chamado de REPL, em inglês: *read*, *eval*, *print*, *loop*. Essa abstração é muito útil para entender o funcionamento do interpretador.

Em resumo, podemos dizer que a etapa *read* traduz o texto para algo que a máquina entende, a etapa *eval* então executa de fato essas instruções, a etapa *print* imprime na tela o resultado da computação (se houver), e a etapa *loop* faz o programa continuar executando.

### *Read*: o tempo de compilação

Quando fornecemos ao Python um trecho de código, a primeira coisa que ele faz, na etapa *read*, é chamar um *parser* para transformar o texto em uma estrutura lógica.

Como parte do *parser*, o código primeiramente passa por uma tokenização feita pelo analisador léxico, processo que quebra o código-fonte em unidades mínimas de significado chamadas de tokens, enquadrando-as em categorias simbólicas.

In [4]:
abc = 1

No código acima, o analisador léxico reconhece que há três tipos de tokens: um nome, de conteúdo ```abc```; um operador, de conteúdo ```=``` (atribuição); e um valor numérico, de conteúdo $1$.

É o analisador léxico que nos diz como podemos, por exemplo, introduzir espaços, quebrar linhas ou indentar o código. Afinal, esses componentes delimitam elementos mínimos de significado. É ele que reconhece as palavras reservadas (*keywords*), os caracteres de escape, as f-strings, os espaços inúteis ou linhas em branco, e o descarte de comentários, e é também ele quem sabe que ```3.``` é um ```float``` enquanto ```3``` é um ```int```.

Posteriormente, a sequência de tokens é passada ao analisador sintático, que é a parte do *parser* capaz de dizer se aquela sequência de tokens, classificados por tipo, obedece ou não à estrutura gramatical da linguagem. Por exemplo, após o comando ```del```, a linguagem espera uma referência a um objeto, mas no caso do operador ```+```, ela espera um objeto não somente depois, mas também antes. Depois de ```def```, espera-se um nome de função, parênteses contendo eventuais argumentos, dois pontos, e um bloco de código, que caso contenha várias linhas lógicas deve ser indentado ou separado por ```;```.

In [ ]:
a = 1 + 2

A expressão acima já é um pouco mais complexa, envolvendo três nomes e dois operadores. Na verdade, na maioria das vezes, o código é muito mais complexo que isso, envolvendo enormes estruturas aninhadas e blocos de controle. A análise sintática dispõe de uma série de técnicas para verificar a corretude de uma sequência de tokens e transformá-la em uma estrutura lógica de onde se pode rastrear a correta ordem de execução do código por parte da máquina. Em CPython especificamente, usa-se a técnica de estruturar o resultado da análise sintática em uma estrutura chamada Árvore Sintática Abstrata (AST), que no caso do exemplo acima seria mais ou menos assim:

In [ ]:
#    =
#   / \
#  a   +
#     / \
#    1   2

Notar que, como pessoas usuárias da linguagem, nós podemos cometer erros tanto léxicos quanto sintáticos. Mas o Python trata todos esses tipos de erro como sintáticos. Nos dois exemplos abaixo, o primeiro erro é léxico (pois não existe token para o caractere ```$``` em Python), enquanto o segundo é sintático (faltando um dois-pontos ```:```), mas o interpretador sempre levanta a exceção ```SyntaxError```:

In [2]:
$

SyntaxError: invalid syntax (1334801704.py, line 1)

In [3]:
if True
    print()

SyntaxError: expected ':' (1297360230.py, line 1)

In [ ]:
# exemplo indentação zuada




TEMPO DE COMPILAÇÃO

Parsing
    |_ analisador léxico -> tokens
    |_ analisador sintático -> parse tree
    |_ conversão em árvore sintática (AST) - CPython
Compilador - AST -> objeto de código compilado

### *Eval*: o tempo de execução

TEMPO DE EXECUÇÃO

VM - aglomerado de estruturas de dados que descreve o estado da execução
    |_ pilha de frames
        |_ ponteiro para o objeto de código compilado (bytecode, tabelas de constantes, nomes e variáveis locais)
        |_ pilha de operandos (resultados intermediários)
        |_ estado da execução (em qual frame está, em qual execução esté, pilha de chamadas)
Interpretador de bytecode - laço que consome a estrutura de dados da VM. gerencia a execução, acionando o runtime para pedir coisas ao sistema operacional conforme as regras de manipulação das pilhas. atua sobre a VM, modificando seu estado interno
Runtime - faz toda a ponte entre a VM/intepretador e o sistema operacional. compreende o runtime environment ou o runtime system, contendo os seguintes elementos:
    |_ heap: objetos criados dinamicamente durante a execução (espaço de armazenamento do runtime)
    |_ gerenciador de memória: mantém tabelas e listas de objetos alocados, faz referência contagem (refcounting), coordena o garbage collector para objetos cíclicos, administra o heap
    |_ garbage collector
    |_ sistema de exceções: rastrear exceções ativas e suas pilhas de traceback
    |_ sistema de namespaces, módulos e importação: gerencia dicionários globais (sys.modules), busca e carrega módulos, compila arquivos .py em bytecode

### *Print* e *loop*: a continuação do *runtime*






Print (imprimir resultado)

O valor retornado da execução é retornado pelo interpretador e exibido na tela.

Não altera a execução em si, mas permite interação dinâmica.

Loop (repetir)

O REPL mantém o mesmo runtime, o mesmo heap e mesmo estado de módulos importados entre linhas.

Variáveis definidas em uma linha permanecem acessíveis na próxima, porque os frames e bindings do módulo principal persistem.

## Desagregando o *bytecode*

## Bônus

Uma curiosidade pouco conhecida sobre as regras de indentação de Python é que ela é implementada usando tokens de ```IDENT``` e ```DEDENT``` pelo analisador léxico e depois analisada pelo analisador sintático.

...







modelo de execução: é apenas a explicação abstrata do que deve fazer o runtime
interpretador CPython: é o executável python3.exe, que dispara tudo isso